# Time series analysis with HyP3 and MintPy

This notebook walks through performing a time-series analysis of the 2019 Ridgecrest, CA earthquake with On Demand InSAR products from the Alaska Satellite facility and MintPy. We'll:

1. Use the [ASF Search Python package](https://docs.asf.alaska.edu/asf_search/basics/) to:
   - Search ASF's catalog for Sentinel-1 SAR products covering the [Ridgecrest](https://earthquake.usgs.gov/storymap/index-ridgecrest.html)
   - Select a reference scene to generate a baseline stack
   - Select a [short baseline subset (SBAS)](https://docs.asf.alaska.edu/vertex/sbas/) of scene pairs for InSAR processing


2. Use the [HyP3 Python SDK](https://hyp3-docs.asf.alaska.edu/using/sdk/) to:
   - Request On Demand InSAR products from ASF HyP3
   - Download the InSAR products when they are done processing


3. Use [GDAL](https://gdal.org/api/index.html#python-api) and [MintPy](https://mintpy.readthedocs.io/en/latest/) to:
   - Prepare the InSAR products for MintPy
   - perform a time-series analysis with MintPy
   
---

**Note:** This notebook does assume you have some familiarity with InSAR processing with MintPy already, and is a minimal example without much context or explanations. If you're new to InSAR and MintPy, I suggest checking out:
* our [InSAR on Demand story map](https://storymaps.arcgis.com/stories/68a8a3253900411185ae9eb6bb5283d3)


* [OpenSARlab's](https://opensarlab-docs.asf.alaska.edu/) highly detailed walkthrough of using HyP3 + MintPy via these notebooks:
  * [Prepare a HyP3 InSAR Stack for MintPy](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/Prepare_HyP3_InSAR_Stack_for_MintPy.ipynb)
  * [MintPy Time-series Analysis](https://nbviewer.org/github/ASFOpenSARlab/opensarlab-notebooks/blob/master/SAR_Training/English/Master/MintPy_Time_Series_From_Prepared_Data_Stack.ipynb)
  
    Note: While these notebooks make some assumptions you're working in OpenSARlab, you can run these 
    notebooks outside OpenSARlab by creating [this conda environment](https://github.com/ASFOpenSARlab/opensarlab-envs/blob/main/Environment_Configs/insar_analysis_env.yml).

## 0. Initial Setup

To run this notebook, you'll need a conda environment with the required dependencies. You can set up a new environment (recommended) and run the jupyter server like:
```shell
conda create -n hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl

conda activate hyp3-mintpy
jupyter notebook hyp3_insar_stack_for_ts_analysis.ipynb
```
Or, install these dependencies into your own environment:
```shell
conda install hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl

jupyter notebook hyp3_insar_stack_for_ts_analysis.ipynb
```

In [ ]:
import os
from pathlib import Path
import glob
from dateutil.parser import parse as parse_date
from osgeo import gdal
import pandas as pd
import asf_search as asf
import hyp3_sdk as sdk

### define the functions

In [ ]:
def get_intersect_rectangle_geotiffs(filelist):
    '''
    :param data_dir: data directory storing the hyp3 products.
    :process get the smallest overlap retangular area to clip the geotiff files.
    :return:
    '''
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in filelist]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]


def prepare_hyp3_product(data_dir):
    filelist = glob.glob(f"{data_dir}/*/*_dem.tif")
    insect_box = get_intersect_rectangle_geotiffs(filelist)
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    list_product_dirs = [f.path for f in os.scandir(data_dir) if f.is_dir()]

    for product_dir in list_product_dirs:
        for file_suffix in files_for_mintpy:
            product_dir = Path(product_dir)
            src_file = product_dir / f'{product_dir.name}{file_suffix}'
            dst_file = product_dir / f'{src_file.stem}_clipped{src_file.suffix}'
            gdal.Translate(destName=str(dst_file), srcDS=str(src_file), projWin=insect_box)


def create_sbas_pairs_file(intersect_point, date_range, sbas_pairs_file):
    '''
    :param intersect_ponit:(longitude, latitude)
    :param date_range: ('yyyy-mm-dd','yyyy-mm-dd')
    :param sbas_pairs_file: file name to store the sbas_pairs
    :return: a set of tuples, each tuple includes a pair of file names
    '''

    point = f'POINT({intersect_point[0]} {intersect_point[1]})'

    search_results = asf.geo_search(
        platform=asf.SENTINEL1,
        intersectsWith=point,
        start=date_range[0],
        end=date_range[1],
        processingLevel=asf.SLC,
        beamMode=asf.IW,
        flightDirection=asf.ASCENDING,
    )

    baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

    columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
    data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

    stack = pd.DataFrame(data, columns=columns)
    stack['startTime'] = stack.startTime.apply(parse_date)

    stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

    sbas_pairs = set()
    for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
        secondaries = stack.loc[
            (stack.sceneName != reference)
            & (stack.temporalBaseline - rt <= max_temporal_baseline)
            & (stack.temporalBaseline - rt > 0)
        ]
        for secondary in secondaries.sceneName:
            sbas_pairs.add((reference, secondary))

    with open(sbas_pairs_file, 'w') as f:
        for t in sbas_pairs:
            f.write(f"{t[0]},{t[1]}\n")

    return sbas_pairs_file


def get_sbas_pairs_set(sbas_pairs_file):
    sbas_pairs = set()
    with open(sbas_pairs_file) as f:
        for line in f:
            line = line.rstrip()
            sbas_pairs.add((line.split(",")[0], line.split(",")[1]))
            
    return sbas_pairs


def download_products_by_sbas_pairs(sbas_pairs_file, project_name, data_dir):
    '''
    :param sbas_pairs: a set of tuples, where each tuple include a pair of
    :proecess: submit the INSAR jobs based on the pairs defined in the sbas_pairs to hyp3 server,
    download the hyp3 products to the data_dir.
    :return:
    '''

    hyp3 = sdk.HyP3(prompt=True)

    jobs = sdk.Batch()
    sbas_pairs = get_sbas_pairs_set(sbas_pairs_file)
    for reference, secondary in sbas_pairs:
        jobs += hyp3.submit_insar_job(reference, secondary, name=project_name,
                                      include_dem=True, include_look_vectors=True)

    jobs = hyp3.find_jobs(name=project_name)

    jobs = hyp3.watch(jobs)

    insar_products = jobs.download_files(data_dir)

    insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]


### set parameters

In [ ]:
project_name = '2019_ridgecrest'
project_home = Path('/media/jzhu4/data/hyp3-mintpy')
work_dir = project_home / project_name
data_dir = work_dir / 'data'

stack_start = parse_date('2019-06-10')
stack_end = parse_date('2019-07-21')
max_temporal_baseline = 12 #days

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    print('Create directory: {}'.format(work_dir))
    
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)
    print('Create directory: {}'.format(data_dir))
    
os.chdir(work_dir)
print('Go to work directory: {}'.format(work_dir))

## 1. Select InSAR pairs with ASF Search

In [ ]:
intersect_point = (-117.599330, 35.769500)

date_range=('2019-06-10', '2019-07-21')

sbas_pairs_file = work_dir / 'sbas_pairs.dat'

sbas_pairs_file = create_sbas_pairs_file(intersect_point, date_range, sbas_pairs_file)

In [ ]:
print(f"sbas pairs are stores at {os.path.abspath(sbas_pairs_file)}")

## 2. Request On Demand InSAR products from ASF HyP3

Use your [NASA Earthdata login](https://urs.earthdata.nasa.gov/) to connect to [ASF HyP3](https://hyp3-docs.asf.alaska.edu/).

In [ ]:
download_products_by_sbas_pairs(sbas_pairs_file, project_name, data_dir)

In [ ]:
print(f"hyp3 products are stored at {os.path.abspath(data_dir)}")

## 3. Time-series Analysis with MintPy

### 3.1 cut off the geotiff files

In [ ]:
prepare_hyp3_product(data_dir)

### 3.2 create the config file

In [ ]:
mintpy_config = work_dir / 'mintpy_config.txt'
mintpy_config.write_text(
f"""
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
##---------geometry datasets:
mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif
""")


### 3.3 run mintpy to do time series analysis

In [ ]:
!smallbaselineApp.py --dir {work_dir} {mintpy_config}

In [ ]:
%matplotlib widget
from mintpy import view, tsview

In [ ]:
view.main([f'{work_dir}/velocity.h5'])

In [ ]:
tsview.main([f'{work_dir}/timeseries.h5'])